# 12장 - 메트릭과 증강을 활용한 훈련 개선

In [ ]:
# 어떻게 측정하고 정량화하며 표현할지, 모델의 작업 수행을 어떻게 개선할지에 대해 설명

## 1절 - 개선을 위한 상위 계획

In [ ]:
# 모델을 일반적인 상황을 학습한다고 가정하고 주요 개념을 구체적으로 보기 위해 문제를 가시적인 용어인 '경비견'과 '새와 도둑'으로 비유
# 11장에서 구현한 내용과 관련된 이슈를 논의할 때 필요한 핵심 개념을 표현하기 위한 '비율: 재현율과 정밀도'라는 도식화된 언어 사용
# 위 개념이 견고해지면 수학적으로 모델의 성능을 평가하고 캡슐화하여 지표를 'F1 점수'라는 하나의 숫자로 압축
# 새로운 메트릭인 F1 점수를 위한 공식을 구현하고 훈련 에포크마다 변화 관찰
# 마지막으로 훈련 결과 개선을 위해 LunaDataset을 '밸런싱'과 '증강'의 관점에서 수정

## 2절 - 착한 개와 나쁜 녀석: 거짓 양성과 거짓 음성

In [1]:
# 거짓 양성(false positive)이란 관심사나 원하는 부류로 분류되는 일이 발생했지만 실제로는 아닌 경우
# 결절이 아니지만 결절로 감지되어 전문의의 확인이 필요한 경우
# 참 양성(true positive)은 관심 항목을 잘 분류한 경우

In [2]:
# 거짓 음성(false negative)란 관심사가 아니거나 관심 부류가 아니라고 판단되는 일이 발생햇지만 실제로는 관심 대상인 경우
# 결절을 발견하지 못한 상황
# 참 음성(true negative)은 관심 대상이 아닌 것을 잘 식별한 경우

In [3]:
# 11장의 모델은 참치 캔이 아닌 모든 것에 대해 야옹거리기를 거부하는 고양이임
# 전체 훈련셋과 검증셋을 평가할 때 정확도 비율에 집중하였으나 좋은 평가 방법이 아님
# 두 경비견에 대한 단편적인 단일 메트릭을 보면 전체 분류 성능을 나타내는 단일 메트릭의 필요성 느껴짐

## 3절 - 긍정과 부정의 경우를 도식화하기

In [4]:
# 두 개의 경계선 존재
# 사람이 동물과 도둑을 구분하는 선과 모델이 샘플을 보고 만들어내는 예측값을 나타내는 경계선

In [5]:
# 그래프로 나타냈을 때 x축은 각 이벤트에 대해 짖을 가치가 있는지의 기준(경비견이 결정), y축은 모호해서 경비견은 인지하기 어렵지만 인간은 인지할 수 있는 경우
# 이진 분류이므로 예측 경계란 단일 숫자로 출력된 값을 우리 분류의 경계값과 비교하는 것으로 생각할 수 있음
# 따라서 분류 경계선을 수직으로 표시

In [6]:
# 진짜 도둑일 가능성은 각기 다르므로 경비견을 각각의 많은 상황 평가해야 함
# 경비견은 x축 부분만 인지 가능하므로 축 주위에는 어지럽게 겹쳐진 이벤트 존재
# 언제 짖을지에 대한 기준을 잡아야 햐지만 경계선이 수직이므로 완벽한 기준을 세우는 것은 불가능함

In [7]:
# 사용할 실제 입력 데이터는 차원이 상당히 높음
# 모델의 역할은 각 이벤트를 비롯해 관련 특성을 해당 사분면의 어딘가로 매핑하고 수직선 하나(분류 경계)로 양성과 음성 분리하는 것
# 위 작업은 모델의 마지막 nn.Linear가 수행
# 수직선의 위치는 classificationThreshold_float와 일치

In [8]:
# 실제로 데이터는 계층을 거치며 차원은 엄청나게 올라가고 출력단에서 다시 한 차원(x축)이 되어 각 샘플당 하나의 스칼라값이 됨
# 여기서 모델이 샘플에서 볼 수 없는 특징을 나타내는 두 번째 차원(y축) 사용하여 결절인지 아닌지의 모호한 경계까지 표현 가능

In [9]:
# 각 사분면 영역과 샘플 수는 모델의 성능을 논의할 때 필요한 값
# 이 값을 사용해 더 복잡한 메트릭을 만들어 현재 객관적으로 얼마나 잘하고 있는지를 판단할 예정
# 각 이벤트 서브셋 사이의 비율을 사용하여 더 나은 메트릭 정의할 예정

#### .1 아무나 보고 짖는 록시의 장점은 재현율

In [10]:
# 재현율(recall, =민감도,sensitivity)은 참 양성과 거짓 음성의 합집합에 대한 양성의 비율
# 재현율 개선을 위해서는 거짓 음성을 줄이면 됨
# 재현율이 1.0에 가깝다는 것은 거짓 양성에 대한 높은 비용을 감수해야 함

#### .2 잠은 많아도 도둑은 잘 잡는 프레스톤의 특기는 정밀도

In [11]:
# 정밀도(precision)는 참 양성과 거짓 양성의 합집합에 대한 참 양성의 비율
# 정밀도 개선을 위해서는 거짓 양성 줄이면 됨
# 정밀도가 1.0에 가깝다는 것은 참 양성의 상당수를 버려야 됨

#### .3 logMetrics로 정밀도와 재현율 구하기

In [12]:
# 정밀도와 재현율은 모델이 어떻게 동작하는지에 대해 중요한 단서를 제공하므로 훈련동안 지켜볼 만한 메트릭
# 둘 중 하나가 0이 되면 모델이 안 좋아진다는 의미
# 이런 상황 파악하면 궤도로 돌아오기 위해 어디를 조사하고 실험해야 할지 확인 가능
# 손실값과 정확도 메트릭에 각 에포크마다 정밀도와 재현율 출력하도록 logMetrics함수 업데이트

In [ ]:
# training.py:315
neg_count = int(negLabel_mask.sum())
pos_count = int(posLabel_mask.sum())

trueNeg_count = neg_correct = int((negLabel_mask & negPred_mask).sum())
truePos_count = pos_correct = int((posLabel_mask & posPred_mask).sum())

falsePos_count = neg_count - neg_correct
falseNeg_count = pos_count - pos_correct

In [13]:
# 코드에서 neg_correct가 trueNeg_count와 같음
# 결절이 아닌 경우 음성 판정에서의 음성이고 식별기가 정확하게 예측했다면 이 경우는 참 음성
# 동일한 방식으로 올바르게 레이블링된 결절 샘플은 참 양성임
# 거짓 양성의 계산은 양성으로 레이블한 수에서 실제 양성인 레이블 수 빼서 남은 값임
# 거짓 음성도 동일하게 계산하되 실제 결절 수 사용

In [ ]:
# precision과 recall 계산하고 metrics_dict에 넣기
# training.py:333
precision = metrics_dict['pr/precision'] = \
            truePos_count / np.float32(truePos_count + falsePos_count)
recall    = metrics_dict['pr/recall'] = \
            truePos_count / np.float32(truePos_count + falseNeg_count)

In [ ]:
# 다중 할당 코드에서 precision과 recall 변수 분리하는 과정이 필수는 아니지만 가독성을 높임

#### .4 궁극의 메트릭: F1 점수

In [14]:
# 상황을 더 유리하게 만들기 위해 두 값을 합성한 무언가가 필요함
# 일반적으로 정밀도와 재현율 합치는 방법인 F1 점수 사용
# F1 점수는 0과 1 사이의 값
# logMetrics이 이 값을 포함하도록 수정

In [ ]:
# training.py:338
metrics_dict['pr/f1_score'] = \
            2 * (precision * recall) / (precision + recall)

In [15]:
# F1 점수는 정밀도와 재현율의 조화 평균으로 산술 평균하는 방식이나 최솟값 취하는 방식보다 사용하기 유용함

In [ ]:
# 각 훈련셋과 검증셋의 로그 출력문에 정밀도, 재현율, F1 점수 모두 넣는 코드
# training.py:341
log.info(
            ("E{} {:8} {loss/all:.4f} loss, "
                 + "{correct/all:-5.1f}% correct, "
                 + "{pr/precision:.4f} precision, "
                 + "{pr/recall:.4f} recall, "
                 + "{pr/f1_score:.4f} f1 score"
            ).format(
                epoch_ndx,
                mode_str,
                **metrics_dict,
            )
        )

In [ ]:
# 각 음성과 양성 샘플에 대해 전체 샘플 수와 정확하게 식별한 경우의 수도 포함
# training.py:353
log.info(
            ("E{} {:8} {loss/neg:.4f} loss, "
                 + "{correct/neg:-5.1f}% correct ({neg_correct:} of {neg_count:})"
            ).format(
                epoch_ndx,
                mode_str + '_neg',
                neg_correct=neg_correct,
                neg_count=neg_count,
                **metrics_dict,
            )
        )

In [ ]:
# 양성인 경우도 위 코드와 거의 유사함

#### .5 새 메트릭으로 모델이 잘 동작하는지 확인하기

In [16]:
# 출력시 경고 메시지를 받으며 계산 값 중 nan도 존재
# 첫째, 훈련셋의 양성 샘플은 실제 양성으로 분류된 케이스가 없어 정밀도와 재현율이 모두 0이며 F1 점수를 구하려다 0으로 나누게 됨
# 둘째, 아무것도 양성으로 분류되지 않아 검증셋에 대한 truePos_count와 falsePos_count가 0이 되어 precision 계산시에 0으로 나누게 됨

In [17]:
# 몇 개의 음성 훈련 샘플은 양성으로 분류됨
# 첫 배치는 랜덤한 결과를 만드므로 첫 배치에서 몇 개의 샘플을 양성으로 분류한 것은 흔한 일임

In [18]:
# 메트릭을 바꾸자 다른 값을 나타내는 것을 보아 메트릭의 근본적 결함을 보여주는 상황 발생

## 4절 - 이상적인 데이터셋의 모습

In [19]:
# 데이터를 밸런싱하기위한 논리적 단계 생성

In [20]:
# 경계값을 움직여 더 나은 결과를 얻는 데에는 양성 클래스와 음성 클래스가 겹치는 구간이 많아 한계가 있음
# 원하는 상황은 레이블의 경계와 분류 경계가 잘 만들어지고 대부분의 샘플이 다이어그램의 양쪽 경계에 집중되어 있는 상황
# 데이터가 쉽게 분류되며 모델이 분류를 수행할 수 있도록 만들어줌
# 모델의 용량은 충분하므로 데이터를 확인

In [21]:
# 데이터는 양성과 음성이 400:1의 비율을 보이며 불균형한 상태
# 대량의 다른 데이터에 뭍혀 잃어버린 상황

#### .1 데이터를 실제보다 '이상'에 가깝게 만들기

In [22]:
# 최선은 더 많은 양성 샘플 확보하는 것
# 훈련의 초기 에포크에서 양성 훈련 샘플이 거의 없으면 사실상 영향 없는 것과 같음

In [23]:
# 신경망의 가중치는 랜덤값으로 초기화되므로 같은 샘플에 대한 신경망의 출력값은 [0.0, 1.0] 범위 내의 랜덤값임
# 예측값이 레이블 결과와 가깝다면 이로 인한 신경망 가중치 변화량은 크지 않지만 정답과 거리가 먼 예측값은 가중치 크게 조정함
# 모델이 랜덤 가중치일 때 출력도 랜덤이므로 약 50만 개의 훈련 샘플에 대해 다음 그룹 가정
# 1. 25만 개의 음성 샘플이 음성으로 예측되어(0.0 ~ 0.5) 음성 예측에 관여하는 신경망의 가중치 조금 변화시킴
# 2. 25만 개의 음성 샘플이 양성으로 예측되어(0.5 ~ 1.0) 음성 예측에 관여하는 신경망의 가중치 크게 변화시킴
# 3. 500 개의 양성 샘플이 음성으로 예측되어(0.0 ~ 0.5) 양성 예측에 관여하는 신경망의 가중치 크금 변화시킴
# 4. 500 개의 양성 샘플이 양성으로 예측되어(0.5 ~ 1.0) 양성 예측에 관여하는 신경망의 가중치 조금 변화시킴

In [24]:
# 1, 4의 크기는 훈련에 거의 영향 주지 않으니 중요하지 않음
# 2, 3이 서로 반대로 당기며 대응하므로 신경망이 퇴보한 상태로 붕괴하지 않도록 예방하기 충분함
# 2는 3보다 500배 크고 배치 크기가 32이므로 500/32 = 15, 즉 15개 배치마다 한 개의 양성 샘플 확인
# 15개 배치 중 14개가 100% 음성 샘플이므로 모델의 가중치는 음성을 예측하도록 유도되어 상황을 악화시킴

In [25]:
# 음성 샘플 수만큼 양성 샘플 늘리면 훈련 초반에는 절반 정도가 잘못 분류될 것임
# 즉 2, 3은 비슷한 크기를 가짐
# 동시에 배치 내에서도 음성과 양성 샘플을 고루 섞는 것을 가정
# 밸런싱은 음성과 양성 사이의 줄다리기 안정화시키며 음성과 양성 사이에 균형을 이룬 배치 내의 조합은 모델에게 두 클래스를 잘 분류할 수 있는 기회를 제공함
# LUNA 데이터에는 소수의 양성 샘플만 존재하므로 훈련 시 반복해서 모델에 노출시킬 예정

In [26]:
# 문제 유형을 파악해서 답을 추측하지 못하도록 참과 거짓 답안을 고루 섞어야 함
# 파이토치를 사용한 배치 시스템은 모델로 하여금 패턴을 파악하서나 사용하지 못하게 해야 함

In [27]:
# 검증에 대해서는 밸런싱 작업을 하지 않음
# 모델을 실세계에서 잘 동작하는 것이 목적이며 실세계는 불균형하기 때문

In [29]:
# DataLoader의 sampler= 옵션 인자로 원래 데이터 순회 순서 무시하고 데이터의 차원 정보나 제한 등을 바꿀 수 있어 통제하에 있지 않은 데이터로 작업할 때 유용한 옵션임
# 공개 데이터셋을 가져와 필요에 따라 형태 바꾸는 방식이 훨씬 편함
# 샘플러를 사용해 다양한 변형을 가하면 데이터셋 캡슐화를 깨뜨리는 단점이 있음

In [30]:
# LunaDataset을 직접 수정해 훈련을 위한 양성과 음성 샘플이 1ㄷ1로 균형 잡히도록 만들 것임
# 음성 훈련 샘플 리스트와 양성 훈련 샘플 리스트를 분리하여 유지하는 상태에서 각각의 리스트에서 넘어오는 샘플을 번갈아 사용
# 모델이 모든 샘플에 대해 '거짓'으로 답해서 좋은 평가 받는 상황 차단
# 음성과 양성 클래스는 서로 섞이므로 이로 인한 가중치 조정의 결과 모델을 결국 두 부류 구별 가능하게 됨

In [ ]:
# ratio_int를 LunaDataset에 더해 N번째 샘플 레이블을 제어하고 레이블로 구분되는 샘플을 관리할 수 있게 하기
# dsets.py:217
class LunaDataset(Dataset):
    def __init__(self,
                 val_stride=0,
                 isValSet_bool=None,
                 series_uid=None,
                 sortby_str='random',
                 ratio_int=0,
                 augmentation_dict=None,
                 candidateInfo_list=None,
            ):
        self.ratio_int = ratio_int

        # 259행
        self.negative_list = [
            nt for nt in self.candidateInfo_list if not nt.isNodule_bool
        ]
        self.pos_list = [
            nt for nt in self.candidateInfo_list if nt.isNodule_bool
        ]

        # 275행
        def shuffleSamples(self): # 매 에포크 시작점에서 이 메소드 호출하여 샘플 순서 렌덤하게 만듦
        if self.ratio_int:
            random.shuffle(self.negative_list)
            random.shuffle(self.pos_list)

In [31]:
# 각 레이블에 대한 전용 리스트 확보함
# 이 리스트를 사용하여 데이터셋의 특정 인덱스에 원하는 레이블 반환하기 쉬움
# ratio_int를 2로 놓아 음성 샘플과 양성 샘플의 비율을 2:1로 만들면 모든 세번째 인덱스는 양성임
# 데이터셋 인덱스를 3으로 나누어 소수점 이하 버리면 양성 인덱스
# 데이터셋 인덱스에서 1 뺀 후 이 값보다 작으면서 제일 가까운 양성 인덱스 빼면 음성 인덱스

In [ ]:
# LunaDataset에 양성 및 음성 인덱스 구하는 코드
# dsets.py:286
def __getitem__(self, ndx):
        if self.ratio_int:
            pos_ndx = ndx // (self.ratio_int + 1)

            if ndx % (self.ratio_int + 1): # 나머지가 0이 아니면 음성 샘플
                neg_ndx = ndx - 1 - pos_ndx
                neg_ndx %= len(self.negative_list) # 오버플로되면 인덱스 앞쪽으로 돌아감
                candidateInfo_tup = self.negative_list[neg_ndx]
            else:
                pos_ndx %= len(self.pos_list) # 오버플로되면 인덱스 앞쪽으로 돌아감
                candidateInfo_tup = self.pos_list[pos_ndx]
        else:
            candidateInfo_tup = self.candidateInfo_list[ndx] # 클래스 밸런싱이 아니면 n번째 샘플 반환

In [32]:
# 비율 낮게 잡으면 데이터셋 다 돌기 전에 양성 샘플을 다 사용하므로 self.pos_list에 인덱싱 전 pos_ndx에 모듈러 연산 적용
# neg_ndx는 음성 샘플 수가 많아 오버플로가 발생하지 않지만 추후 변경으로 발생할 수 있으므로 댑하기 위해 모듈러 연산 적용

In [ ]:
# 개별 에포크의 속도를 높이기 위해 데이터셋의 길이도 변경: __len__ 값을 200000으로 하드코딩
# dsets.py:280
def __len__(self):
        if self.ratio_int:
            return 200000
        else:
            return len(self.candidateInfo_list)

In [33]:
# 더 이상 샘플 수에 얽매이지 않게 되었고 '전체 에포크' 개념도 밸런싱을 윙해 양성 샘플을 훈련셋에 반복 노출하므로 무의미하게 됨
# 20만 개의 샘플을 사용하므로 훈련 실행 후 결과 확인까지의 시간도 줄어들음(빠른 피드백은 좋음)
# 에포크마다 샘플 수도 정리했으니 편리하게 에포크의 길이도 조절 가능

In [ ]:
# 명령행 파라미터까지 넣어 완벽하게 만들기
# training.py:31
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        parser.add_argument('--balanced',
            help="Balance the training data to half positive, half negative.",
            action='store_true',
            default=False,
        )

In [ ]:
# 파라미터는 LunaDataset 생성자에 전달됨
# training.py:137
def initTrainDl(self):
        train_ds = LunaDataset(
            val_stride=10,
            isValSet_bool=False,
            ratio_int=int(self.cli_args.balanced), # True가 1로 변환됨
            augmentation_dict=self.augmentation_dict,
        )

#### .2 균형잡힌 LunaDataset으로 훈련시킨 결과의 차이

In [34]:
# --balanced로 돌린 결과는 전과 비교할 수 없을만큼 좋아짐
# 음성 샘플에 대한 정확도 5% 손해보고 양성에 대한 정확도 86% 얻음
# 양성 샘플에 비해 음성 샘플이 400배 많으므로 1%만 잘못 분류되어도 양성 샘플이 있는 경우의 4배나 많이 잘못 분류된 것
# 추가적인 에포크 훈련 가동

In [35]:
# val_mal XX.X% correct 수치가 에포크 2에서 87.5%, 에포크 5에서 92.6%, 에포크 20에서 86.8%로 다시 낮아짐
# 음성 훈련 샘플은 98.8%, 양성 샘플은 99.1%가 정확한 것의 의미 확인

#### .3 과적합 증상 알아채기

In [36]:
# 전형적인 과적합 현상
# 양성 샘플에 대한 손실값 그래프를 보면 양성 샘플 훈련 손실이 거의 0에 가까워지지만 검증 손실은 점점 증가함
# 모델이 더 이상 개선되지 않으므로 훈련 스크립트 실행을 중단하는 편이 좋음

In [37]:
# 이 경향이 양성 손실값에서만 나타나지만 전체 손실에서는 문제 없어 보임
# 검증셋이 균형잡히지 않았기 때문에 전체 손실갑싱 음성 샘플에 의해 좌지우지됨
# 음성 샘플을 400배나 많이 가지기 때문에 개별 샘플에 대한 세부 내용 기억이 어렵기 때문에 음성 손실값의 경향은 훌륭함
# 양성 훈련셋은 1215개의 샘플만 있어 모델이 샘플을 기억해버리는 걸 방해하지 못함

In [ ]:
# 단 양성 검증셋의 70% 정도는 구별하므로 일반화는 어느 정도 진행되고 있음
# 모델의 훈련 방법만 바꾸어 훈련셋과 검증셋 모두에 대해 옳은 방향으로 진행시키기

## 5절 - 과적합 문제 다시 살펴보기

In [ ]:
# 모델 훈련의 목적은 모델이 우리가 관심 있어 하는 부류의 일반적 특성을 데이터셋을 보고 배우게 하는 것
# 과적합이 발생하면 모델은 일반화하는 능력 잃어버림

## 6절 - 데이터 증강으로 과적합 방지하기

In [39]:
# 개별 샘플에 대한 합성을 통해 원래보다 많은 새로운 데이터셋을 만들어 데이터셋을 증강(augment)시키고 훈련 효과 증가시키기
# 증강을 통해 원본 샘플에 있는 클래스가 가진 일반적 특성을 동일하게 유지한 합성 샘플 만들어 모델이 암기하지 못하게 만듦
# 증강이 잘 되면 모델이 암기할 수 없어지고 일반화에 점점 의존하게 됨
# 증강은 데이터가 제한적일 경우 특히 유용함
# 모든 증강이 효과적이지는 않음

#### .1 특별 데이터 증강 기술

In [40]:
# 5가지 특별한 데이터 증강 기술 확인
# 하나 혹은 모두를 적용할 수도 있고 개별 적용하거나 중첩할 수도 있음
# 상하 혹은 좌우 반전, 평행이동, 확대 및 축소, 회전, 노이즈 추가
# 원본 샘플과는 다르지만 훈련에는 유용하도록 훈련 샘플 본연의 특징은 유지되어야 함

In [ ]:
# 원본 CT 데이터 덩어리를 가져와 수정할 함수로 getCtAugmentedCandidate 정의
# 아핀 변환 행렬을 정의하고 파이토치의 affine_grid와 grid_sample을 함께 사용해 후보 샘플 생성
# dsets.py:149
def getCtAugmentedCandidate(
        augmentation_dict,
        series_uid, center_xyz, width_irc,
        use_cache=True):
    if use_cache:
        ct_chunk, center_irc = \
            getCtRawCandidate(series_uid, center_xyz, width_irc)
    else:
        ct = getCt(series_uid)
        ct_chunk, center_irc = ct.getRawCandidate(center_xyz, width_irc)

    ct_t = torch.tensor(ct_chunk).unsqueeze(0).unsqueeze(0).to(torch.float32)

In [ ]:
# 먼저 직접 CT를 로딩하거나 캐시로부터 ct_chunk를 얻은 후 텐서로 변환
# 아래는 아핀 그리드와 샘플링 코드
# dsets.py:162
transform_t = torch.eye(4)

# ... # 이 쯤에서 transform_tensor 수행
# 195행
affine_t = F.affine_grid(
            transform_t[:3].unsqueeze(0).to(torch.float32),
            ct_t.size(),
            align_corners=False,
        )

augmented_chunk = F.grid_sample(
            ct_t,
            affine_t,
            padding_mode='border',
            align_corners=False,
        ).to('cpu')

# 214행
return augmented_chunk[0], center_irc

In [41]:
# 추가적인 작업을 하지 않으면 위 함수는 아무 변화도 만들지 않음
# 추가할 실제 변환 확인

In [ ]:
# 미러링
# 샘플 미러링할 때 픽셀값은 그대로 두고 이미지의 오리엔테이션만 변경
# 좌우, 앞뒤 방향은 종양의 성장에 강한 상관관계가 없으므로 샘플을 대표하는 특성을 바꾸지 않고 이미지 반전시킬 수 있어야 함
# dsets.py:165
for i in range(3):
    if 'flip' in augmentation_dict:
        if random.random() > 0.5:
            transform_t[i,i] *= -1

In [42]:
# grid_sample 함수는 [-1, 1] 범위를 이전 텐서와 새 텐서 모두에 매핑(사이즈 다를 경우 암묵적으로 비율 조정)
# 범위 매핑으로 데이터를 미러링하기 위해 변환 행렬의 관련 요소에 -1 곱하면 됨

In [ ]:
# 랜덤 크기만큼 평행이동
# 컨볼루션은 평행이동에 독립적이므로 분류 성능에는 큰 차이 주지 못함
# 오프셋이 복셀 단위의 정수가 아닌 경우에 더 두드러진 차이 만들 수 있음
# 삼중 선형 보간법(trilinear interpolation)을 사용해 약간의 블러 처리 들어가는 형태로 데이터 다시 샘플링
# 샘플의 경계에 있는 복셀은 반복되므로 경계를 따라 얼룩진 줄무늬처럼 보이게 됨
# dsets.py:170
for i in range(3):
    if 'offset' in augmentation_dict:
        offset_float = augmentation_dict['offset']
        random_float = (random.random() * 2 - 1)
        transform_t[i,3] = offset_float * random_float

In [43]:
# offset 파라미터는 그리드 샘플 함수가 기대하는 [-1, 1] 범위와 같은 비율로 표시되는 최대 오프셋

In [ ]:
# 확대 축소
# 이미지 크기 살짝 확대축소하는 것은 미러링이나 평행이동과 유사함
# 경계 복셀이 반복되는 효과 가짐
# dsets.py:175
for i in range(3):
    if 'scale' in augmentation_dict:
        scale_float = augmentation_dict['scale']
        random_float = (random.random() * 2 - 1)
        transform_t[i,i] *= 1.0 + scale_float * random_float

In [45]:
# random_float는 [-1, 1] 범위로 변환되므로 scale_float * random_float를 더하거나 1.0에서 빼는 것에 영향받지 않음

In [46]:
# 회전
# 샘플 변환 시 원래 샘플이 표현하는 특성을 해치지 않도록 주의해야 하는데 현재 샘플의 행, 열(x, y축)은 간격이 균일하지만 인덱스 방향(z축)은 다르므로 호환되지 않음
# 인덱스 축은 특별한 것으로 간주하고 회전을 x-y축으로만 고정
# dsets.py:181
if 'rotate' in augmentation_dict:
        angle_rad = random.random() * math.pi * 2
        s = math.sin(angle_rad)
        c = math.cos(angle_rad)

        rotation_t = torch.tensor([
            [c, -s, 0, 0],
            [s, c, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
        ])

        transform_t @= rotation_t

In [ ]:
# 노이즈
# 앞에의 방법들과는 다르게 샘플을 어느 정도 망가뜨리므로 노이즈를 너무 많이 넣으면 실 데이터를 분류하지 못하게 만들 수 있음
# 평행이동이나 확대축소도 극단적인 값 사용하면 마찬가지일 수 잇지만 샘플의 경계에만 영향을 미치는 값을 사용한 반면 노이즈는 이미지 전체에 영향 미침
# dsets.py:208
if 'noise' in augmentation_dict:
        noise_t = torch.randn_like(augmented_chunk)
        noise_t *= augmentation_dict['noise']

        augmented_chunk += noise_t

In [ ]:
# 이전의 증간 기술은 데이터셋의 크기를 키운 반면 노이즈는 모델의 분류를 어렵게 만듦

In [ ]:
# 증강된 후보 살펴보기
# __getitem__을 호출할 때마다 증강된 데이터셋에 다시 랜덤하게 증강 적용함

#### .2 데이터 증강 효과 확인하기

In [ ]:
# 증강별로 모델 훈련하고 모든 증강 타입 섞은 데이터로 훈련한 후 텐서보드로 수치 확인
# 각 증강 타입을 키고 끄기 위해 augmentation_dict 생성 부분을 명령행에 노출할 필요 있음
# 프로그램 인자는 parser.add_argument 호출을 통해 추가하여 augmentation_dict 만듦
# training.py:105
self.augmentation_dict = {}
if self.cli_args.augmented or self.cli_args.augment_flip:
    self.augmentation_dict['flip'] = True
if self.cli_args.augmented or self.cli_args.augment_offset:
    self.augmentation_dict['offset'] = 0.1 # 이 값은 적당한 영향 주기 위해 경험에서 나온 것이며 더 좋은 값 존재 가능
if self.cli_args.augmented or self.cli_args.augment_scale:
    self.augmentation_dict['scale'] = 0.2 # 이 값은 적당한 영향 주기 위해 경험에서 나온 것이며 더 좋은 값 존재 가능
if self.cli_args.augmented or self.cli_args.augment_rotate:
    self.augmentation_dict['rotate'] = True
if self.cli_args.augmented or self.cli_args.augment_noise:
    self.augmentation_dict['noise'] = 25.0 # 이 값은 적당한 영향 주기 위해 경험에서 나온 것이며 더 좋은 값 존재 가능

In [47]:
# 명령행 인자가 준비되었으므로 실행
# 실행하는 동안 텐서보드 띄울 수 있음

In [48]:
# tag: correct/all 그래프 보면 개별 증강 유형에 대한 그래프가 뒤섞여 잇음
# 모든 증강 유형 사용할 때와 사용하지 않을 때의 그래프가 뒤섞인 개별 증강 유형 그래프의 위 아래에 대칭적으로 놓임
# 즉 증강 타입 섞었을 때가 각각의 합보다 효과가 크다는 의미
# 하지만 전체 증강 유형을 섞은 경우 오답률 증가
# 일반적으로는 나쁘지만 모든 증강 다 사용할 때가 양성 후보 샘플 찾아내는데 훨씬 나아 보임
# 모든 증강 사용한 모델의 재현율은 꽤 훌륭하며 과적합 면에서도 좋아짐

In [49]:
# 노이즈 증강 사용한 모델은 원래보다 결절 식별 능력 감소함
# 모델의 분류를 어렵게 만드는 증강이라는 맥락에서 이해 가능

In [50]:
# 회전 증강을 사용한 모델이 재현율과 정밀도 면에서 전체 증강 사용한 모델만큼 좋다는 점
# F1 점수도 제한적인 점은 있지만 회전 증강에서 더 나은 점수 보여줌

In [51]:
# 현 프로젝트에서는 높은 재현율을 요구하므로 전체 증강 사용한 모델 채택
# 적절한 에포크는 F1 점수 이용해 고르기
# 실제 프로젝트에서는 어떤 증강 타입 골라 쓸지, 어떤 파라미터값을 사용해야 할지 추가적인 고민 할 수 있음

## 7절 - 결론

In [52]:
# 모델 성능 재해석
# 어설픈 평가로는 쉽게 잘못 판단할 수 있으며 모델을 잘 평가하기 위해서는 사용하는 지표에 대한 직관적인 이해와 감이 필요함
# 이런 기본적인 부분 내재화 하면 프로젝트 중 헤매도 빠르게 알아차릴 수 있음

In [53]:
# 불충분한 데이터 소스 다루는 법 확인
# 특성 보존된 훈련 샘플 합성하는 것은 매우 유용
# 실제로 훈련을 위한 데이터를 충분히 확보하는 경우는 매우 드뭄

## 8절 - 연습 문제

In [54]:
# 1. F1 점수는 1외의 값도 지원하도록 일반화할 수 있다.
#   a. https://en.wikipedia.org/wiki/F-score 읽고 F2 점수와 F0.5 점수 구현하라
#   b. F1, F2, F0.5 중 어떤 것이 이 프로젝트에서 가장 사용할 만한 것인지 알아보라. 값을 추적해서 F1 점수와 비교 대조해보라

In [55]:
# 2. WeightedRandonSampler 방식을 구현하여 LunaDataset을 ratio_int=0으로 양성과 음성 샘플을 밸런싱하라.
#   a. 각 샘플 클래스에 대해 필요한 정보는 어떻게 확보했는가?
#   b. 어떤 접근 방식이 쉬운가? 어떤 방식의 코드가 가독성이 좋은가?

In [56]:
# 3. 여러 클래스 밸런싱 전략으로 실험해보자.
#   a. 2에포크를 지났을 때 제일 좋은 결과를 보이는 비율은? 20에포크를 지났을 때는?
#   b. 만일 비율이 epoch_ndx 함수라면?

In [57]:
# 4. 여러 데이터 증강 유형을 실험해보자.
#   a. 각 접근 방법을 더 공격적으로 만들 수 있을까(노이즈나 오프셋 등)?
#   b. 노이즈 증강을 쓰면 훈련 결과가 좋아지는가, 방해받는가?
#       - 결과를 바꿀만한 다른 값이 있나?
#   c. 다른 프로젝트에서 사용하는 데이터 증강에 대해 찾아보라. 그중에 우리가 사용할 만한 것이 있는가?
#       - 양성 결절 후보에 대해 믹스업(mixup) 증강을 구현하고 도움이 되는지 확인하라.

In [58]:
# 5. nn.BatchNorm의 초기 정규화를 바꿔서 모델을 재훈련시키자.
#   a. 고정된 정규화로 더 나은 결과를 얻을 수 있나?
#   b. 어떤 정규화 오프셋과 비율 값이 효과가 있나?
#   c. 제곱근 같은 비선형 정규화가 도움되는가?

In [ ]:
# 6. 이 책에서 다루지 않은 텐서보드가 제공하는 다른 그래프나 정보는 어떤 것이 있나?
#   a. 신경망의 가중치에 대한 정보를 표시할 수 있는가?
#   b. 특정 샘플에 대한 모델 실행에 대해 중간 결과를 볼 수 있는가?
#       - nn.Sequential 객체로 래핑된 모델의 백본인 경우, 도움이 되는가 혹은 반대인가?